![cyber_photo](cyber_photo.jpg)

Cyber threats are a growing concern for organizations worldwide. These threats take many forms, including malware, phishing, and denial-of-service (DOS) attacks, compromising sensitive information and disrupting operations. The increasing sophistication and frequency of these attacks make it imperative for organizations to adopt advanced security measures. Traditional threat detection methods often fall short due to their inability to adapt to new and evolving threats. This is where deep learning models come into play.

Deep learning models can analyze vast amounts of data and identify patterns that may not be immediately obvious to human analysts. By leveraging these models, organizations can proactively detect and mitigate cyber threats, safeguarding their sensitive information and ensuring operational continuity.

As a cybersecurity analyst, you identify and mitigate these threats. In this project, you will design and implement a deep learning model to detect cyber threats. The BETH dataset simulates real-world logs, providing a rich source of information for training and testing your model. The data has already undergone preprocessing, and we have a target label, `sus_label`, indicating whether an event is malicious (1) or benign (0).

By successfully developing this model, you will contribute to enhancing cybersecurity measures and protecting organizations from potentially devastating cyber attacks.


### The Data

| Column     | Description              |
|------------|--------------------------|
|`processId`|The unique identifier for the process that generated the event - int64 |
|`threadId`|ID for the thread spawning the log - int64|
|`parentProcessId`|Label for the process spawning this log - int64|
|`userId`|ID of user spawning the log|Numerical - int64|
|`mountNamespace`|Mounting restrictions the process log works within - int64|
|`argsNum`|Number of arguments passed to the event - int64|
|`returnValue`|Value returned from the event log (usually 0) - int64|
|`sus_label`|Binary label as suspicous event (1 is suspicious, 0 is not) - int64|

More information on the dataset: [BETH dataset](accreditation.md)

In [27]:
# Import required libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as functional
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from torchmetrics import Accuracy
# from sklearn.metrics import accuracy_score  # uncomment to use sklearn

In [28]:
# Load preprocessed data
train_df = pd.read_csv('labelled_train.csv')
test_df = pd.read_csv('labelled_test.csv')
val_df = pd.read_csv('labelled_validation.csv')

# View the first 5 rows of training set
train_df.head()

,processId,threadId,parentProcessId,userId,mountNamespace,argsNum,returnValue,sus_label
0,381,7337,1,100,4026532231,5,0,1
1,381,7337,1,100,4026532231,1,0,1
2,381,7337,1,100,4026532231,0,0,1
3,7347,7347,7341,0,4026531840,2,-2,1
4,7347,7347,7341,0,4026531840,4,0,1


In [29]:
# Start coding here
# Use as many cells as you need

In [30]:
#Separating features and labels of the training, testing and validation datasets
x_train_df=train_df.drop("sus_label",axis=1).values
y_train_df=train_df["sus_label"].values
x_test_df=test_df.drop("sus_label", axis=1).values
y_test_df=test_df["sus_label"].values
x_val_df= val_df.drop("sus_label",axis=1).values
y_val_df= val_df["sus_label"].values

In [31]:
# Using Standard scaler to scale the features from 0 to 1 to ensure gradient stability
scale=StandardScaler()

#training features
x_train_scaled=scale.fit_transform(x_train_df)

#testing features
x_test_scaled=scale.transform(x_test_df)

#validation features
x_val_scaled=scale.transform(x_val_df)

In [32]:
# converting the data into torch tensors
x_train_tensor=torch.tensor(x_train_scaled,dtype=torch.float32)
y_train_tensor=torch.tensor(y_train_df, dtype=torch.float32).view(-1,1)
x_test_tensor=torch.tensor(x_test_scaled, dtype=torch.float32)
y_test_tensor=torch.tensor(y_test_df, dtype=torch.float32).view(-1,1)
x_val_tensor=torch.tensor(x_val_scaled, dtype=torch.float32)
y_val_tensor=torch.tensor(y_val_df, dtype=torch.float32).view(-1,1)

In [33]:
# defining the model layers, this is a binary classification, hence the sigmoid activation is used
model=nn.Sequential(
    nn.Linear(x_train_tensor.shape[1], 128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,1),
    nn.Sigmoid()
)

In [34]:
# Initialising the loss function and the optimiser
criterion = nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), lr=0.001, momentum=0.97)

In [35]:
# Training loop 
num_epoch=10
for epoch in range (num_epoch):
    model.train()    # for training the model
    optimizer.zero_grad()    # making the gradient zero for the start
    outputs = model(x_train_tensor)     # Forward pass: compute the output of the model
    loss = criterion(outputs, y_train_tensor)    # the difference between the predicted and observed y value
    loss.backward()    # Backward pass to compute new gradients for the layers. Computes the contribution of each parameter to the loss
    optimizer.step()     # Update the gradient fields
    
    

In [36]:
# Evaluation of the model
model.eval()
with torch.no_grad():
    y_train_pred = model(x_train_tensor).round()
    y_test_pred = model(x_test_tensor).round()
    y_val_pred = model(x_val_tensor).round()

# Calculating accuracy of the model with torchmetrics
acc = Accuracy(task = "binary")
acc_train = acc(y_train_pred,y_train_tensor)
acc_test = acc(y_test_pred, y_test_tensor)
acc_val = acc(y_val_pred, y_val_tensor)

# convert to int or float
train_accuracy = acc_train.item()
test_accuracy = acc_test.item()
val_accuracy = acc_val.item()

print("Training accuracy: {0}".format(train_accuracy))
print("Validation accuracy: {0}".format(val_accuracy))
print("Testing accuracy: {0}".format(test_accuracy))



Training accuracy: 0.9957845211029053
Validation accuracy: 0.9989045858383179
Testing accuracy: 0.9439319968223572
